In [3]:
!pip install datasets


     ------------------------------------ 494.8/494.8 kB 373.6 kB/s eta 0:00:00
     ------------------------------------ 134.8/134.8 kB 568.9 kB/s eta 0:00:00
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached requests-2.32.4-py3-none-any.whl (64 kB)
     -------------------------------------- 515.3/515.3 kB 1.2 MB/s eta 0:00:00
     -------------------------------------- 193.6/193.6 kB 1.3 MB/s eta 0:00:00
     ------------------------------------ 451.3/451.3 kB 940.5 kB/s eta 0:00:00
     -------------------------------------- 116.3/116.3 kB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 43.8/43.8 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 41.5/41.5 kB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 46.0/46.0 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 86.8/86.8 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.1
    Uninstalling t

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.27 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.27 requires nbformat==5.4.0, but you have nbformat 5.7.0 which is incompatible.
conda-repo-cli 1.0.27 requires requests==2.28.1, but you have requests 2.32.4 which is incompatible.


In [10]:
!pip install pdfplumber


In [12]:
from datasets import load_dataset
import pandas as pd

# Load dataset and remove the problematic 'pdf' column
dataset = load_dataset("shrijayan/gov_myscheme", split="train", trust_remote_code=True)
dataset = dataset.remove_columns(['pdf'])  # This prevents pdf decoding

# Convert to DataFrame
df = pd.DataFrame(dataset)

# Preview
print(df.columns)
df.head()


`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'shrijayan/gov_myscheme' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


Index([], dtype='object')


""


In [13]:
import os
import pandas as pd

# Path to your Andhra text files
data_path = "Indian_govt_schemes/andhra-pradesh"

# Result list
data = []

for filename in os.listdir(data_path):
    if filename.endswith(".txt"):
        with open(os.path.join(data_path, filename), 'r', encoding='utf-8') as f:
            content = f.read()

            # --- Basic Cleaning ---
            lines = content.splitlines()
            lines = [line.strip() for line in lines if line.strip()]

            # --- Extract Fields ---
            scheme_name = lines[0] if lines else "Unknown"
            
            # Use keyword matches to extract summaries
            eligibility_lines = [line for line in lines if "eligible" in line.lower() or "income" in line.lower()]
            benefit_lines = [line for line in lines if "benefit" in line.lower() or "support" in line.lower() or "reimbursement" in line.lower()]

            eligibility_summary = " ".join(eligibility_lines[:2]) if eligibility_lines else ""
            benefit_summary = " ".join(benefit_lines[:2]) if benefit_lines else ""

            # Store the results
            data.append({
                "State": "Andhra Pradesh",
                "Filename": filename,
                "Scheme Name": scheme_name,
                "Eligibility": eligibility_summary,
                "Benefit": benefit_summary,
                "Raw Content": content
            })

# Convert to CSV
df = pd.DataFrame(data)
df.to_csv("andhra_schemes_extracted.csv", index=False)
print("✅ Extraction complete. Saved to andhra_schemes_extracted.csv")


✅ Extraction complete. Saved to andhra_schemes_extracted.csv


In [14]:
import os
import re
import pandas as pd

def extract_fields(text):
    lines = text.splitlines()
    lines = [line.strip() for line in lines if line.strip() and "Table of Contents" not in line]

    # Scheme Name
    scheme_name = next((line for line in lines if re.search(r'scheme|yojana|vidya|mission', line, re.IGNORECASE)), "Unknown")

    # Eligibility
    eligibility_lines = [line for line in lines if re.search(r'eligible|income|limit|category|bpl|poverty', line, re.IGNORECASE)]
    eligibility_summary = " ".join(eligibility_lines[:2]) if eligibility_lines else ""

    # Benefit
    benefit_lines = [line for line in lines if re.search(r'benefit|support|reimbursement|financial aid|money|help', line, re.IGNORECASE)]
    benefit_summary = " ".join(benefit_lines[:2]) if benefit_lines else ""

    return scheme_name, eligibility_summary, benefit_summary, "\n".join(lines)

# Main processing
root_dir = "Indian_govt_schemes"  # Main folder with state subfolders
all_data = []

for state_folder in os.listdir(root_dir):
    state_path = os.path.join(root_dir, state_folder)
    if os.path.isdir(state_path):
        for filename in os.listdir(state_path):
            if filename.endswith(".txt"):
                file_path = os.path.join(state_path, filename)
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                    scheme_name, eligibility, benefit, raw = extract_fields(content)

                    all_data.append({
                        "State": state_folder.replace("-", " ").title(),
                        "Filename": filename,
                        "Scheme Name": scheme_name,
                        "Eligibility": eligibility,
                        "Benefit": benefit,
                        "Raw Content": raw
                    })

# Save to CSV
df_all = pd.DataFrame(all_data)
df_all.to_csv("all_states_schemes_cleaned.csv", index=False)
print("✅ Saved all state schemes to all_states_schemes_cleaned.csv")


✅ Saved all state schemes to all_states_schemes_cleaned.csv
